In [16]:
import sys
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import pandas as pd
import json

In [17]:
import numpy as np
import copy

In [18]:
#before
base_path = '.'
df_b = pd.read_pickle(f'{base_path}/av_base.pk')


# after q50_over_avg
d_path = '/raid/AISSEL/htest/datasets/av_data/ftest/missed_q50_over_avg'
df_f = pd.read_pickle(f'{d_path}/av_mq50_over_avg.pk')

# wit
d_path = '/raid/AISSEL/htest/datasets/av_data/wit/missed_q50_over_avg'
df_w = pd.read_pickle(f'{d_path}/av_mq50_over_avg.pk')

#random
d_path = '/raid/AISSEL/htest/datasets/av_data/wit/random_missed'
df_r = pd.read_pickle(f'{d_path}/av_mq50_over_avg.pk')

In [19]:
annotation_file = './captions_val2014.json'
with open(annotation_file, 'r') as f:
    jdata = json.load(f)

In [20]:
def id2imdf(_df):
    _df = _df[['uniq_id']]
    _df['license'] = 1
    _df['url'] = 'http://farm9.staticflickr.com'
    _df['file_name'] = 'test.jpg'
    _df['width'] = 640
    _df['height'] = 360
    _df['date_captured'] = '2013-11-14 11:18:45'
    _df.columns = _df.columns.str.replace('uniq_id', 'id')
    return _df

In [21]:
def h_gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
        caption.append(row['h_caption'])
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [22]:
def concat_gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
        caption.append(row['gt'])
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [23]:
def gt2anno(_df):
    image_id = []
    caption = []
    for index, row in _df.iterrows():
#         print(row['image_id'], row['gt'])
        for el in row['tag_list']:
            image_id.append(row['image_id'])
            caption.append(el)
#         break
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(caption))
    df_['caption'] = caption
    return df_


In [24]:
def web_2anno(_df):
    image_id = []
#     caption = []
    for index, row in _df.iterrows():
        image_id.append(row['image_id'])
#         caption.append('pedestrian')
    
    df_ = pd.DataFrame()
    df_['image_id'] = image_id
    df_['id'] = np.arange(len(image_id))
    df_['caption'] = _df['caption'].to_list()
    return df_

In [25]:
def create_dict(df_wp):
#     df_wp = pd.read_pickle(df_wp)
    df_n = df_wp.drop_duplicates(subset='uniq_id', keep="last")
    df_n = df_n.reset_index(drop=True)
    
    gk = df_wp.groupby('topics')
    pg_dict = dict()
    for el in gk.groups.keys():
        pg_dict[el] = dict()
        pg_dict[el]['data'] = gk.get_group(el)
    #     pg_dict[el]['data'] = pg_dict[el]['data'][pg_dict[el]['data']['topic2caption_sim'] >= pg_dict[el]['data']['topic2caption_sim'].quantile(.25)]
        pg_dict[el]['data'] = pg_dict[el]['data'].reset_index(drop=True)
        temp = pg_dict[el]['data'][['image_id', 'ofa_caption']]
        temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
        pg_dict[el]['results'] = temp
        pg_dict[el]['images'] = id2imdf(pg_dict[el]['data'])
    #     pg_dict[el]['split_annotations'] = gt2anno(pg_dict[el]['data'])
        pg_dict[el]['human_annotations'] = h_gt2anno(pg_dict[el]['data'])
        pg_dict[el]['con_annotations'] = concat_gt2anno(pg_dict[el]['data'])
        pg_dict[el]['web_annotations'] = web_2anno(pg_dict[el]['data'])
    return df_n, pg_dict

In [26]:
def crate_json(pg_dict, el, t='human_annotations'):
    t_data = dict()
    t_data['info'] = jdata['info']
    t_data['images'] = pg_dict[el]['images'].to_dict('records')
    t_data['type'] = jdata['type']
    t_data['licenses'] = jdata['licenses']
    t_data['annotations'] = pg_dict[el][t].to_dict('records')
    return t_data
    

In [27]:
def file_path_dict(pg_dict, test='wit'):
    file_dict = dict()
    for el in pg_dict:
        file_dict[el] = dict()
#         print(el)

        s_data = crate_json(pg_dict, el,t='human_annotations')
        c_data = crate_json(pg_dict, el,t='con_annotations')  
        o_data = crate_json(pg_dict, el,t='web_annotations')  
        res = pg_dict[el]['results'].to_dict('records')

        file_dict[el]['results'] = f"data/{test}_{el}_results.json"
        with open(file_dict[el]['results'], "w") as final:
            json.dump(res, final)

        file_dict[el]['sval'] = f"data/{test}_{el}_sval.json"
        with open(file_dict[el]['sval'], "w") as final:
            json.dump(s_data, final)

        file_dict[el]['cval'] = f"data/{test}_{el}_cval.json"
        with open(file_dict[el]['cval'], "w") as final:
            json.dump(c_data, final)

        file_dict[el]['oval'] = f"data/{test}_{el}_oval.json"
        with open(file_dict[el]['oval'], "w") as final:
            json.dump(o_data, final)
    return file_dict

In [28]:
def calc_metrics(file_dict, topic='a', anno='s'):
    if anno == 's':
        annotation_file = file_dict[topic]['sval']
    elif anno == 'c':
        annotation_file = file_dict[topic]['cval']
    else:
        annotation_file = file_dict[topic]['oval']
        
    results_file = file_dict[topic]['results']
    coco = COCO(annotation_file)
    coco_result = coco.loadRes(results_file)
    coco_eval = COCOEvalCap(coco, coco_result)
    coco_eval.params['image_id'] = coco_result.getImgIds()
    coco_eval.evaluate()
    return coco_eval.eval


# Base

In [29]:
df_b1, pg_dict_b = create_dict(df_b)

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [30]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_b1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [31]:
t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)

In [32]:
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

In [33]:
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4341, 'reflen': 7635, 'guess': [4341, 3866, 3391, 2916], 'correct': [655, 82, 13, 3]}
ratio: 0.5685658153240906
Bleu_1: 0.071
Bleu_2: 0.026
Bleu_3: 0.011
Bleu_4: 0.005
computing METEOR score...
METEOR: 0.035
computing Rouge score...
ROUGE_L: 0.090
computing CIDEr score...
CIDEr: 0.108
computing SPICE score...
SPICE: 0.050


In [34]:
file_dict_b = file_path_dict(pg_dict_b, test='base')

In [35]:
web_res_dict = dict()
for el in pg_dict_b:
    web_res_dict[el] = calc_metrics(file_dict_b, topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 108, 'reflen': 128, 'guess': [108, 96, 84, 72], 'correct': [11, 2, 0, 0]}
ratio: 0.8437499999934083
Bleu_1: 0.085
Bleu_2: 0.038
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.039
computing Rouge score...
ROUGE_L: 0.076
computing CIDEr score...
CIDEr: 0.046
computing SPICE score...
SPICE: 0.127
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 495, 'reflen': 822, 'guess': [495, 439, 383, 327], 'correct': [105, 12, 1, 0]}
ratio: 0.6021897810211653
Bleu_1: 0.110
Bleu_2: 0.039
Bleu_3: 0.013
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.054
computing R

In [36]:
all_res_b = []
for el in pg_dict_b:
    all_res_b.append(web_res_dict[el])
df_res = pd.DataFrame(all_res_b)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.08,0.03,0.01,0.00,0.04,0.08,0.10,0.05
std,0.03,0.02,0.01,0.00,0.01,0.03,0.06,0.04
min,0.02,0.01,0.00,0.00,0.02,0.04,0.03,0.01
25%,0.05,0.01,0.00,0.00,0.03,0.06,0.05,0.02
50%,0.08,0.04,0.00,0.00,0.03,0.08,0.08,0.03
75%,0.10,0.04,0.01,0.00,0.04,0.11,0.15,0.07
max,0.14,0.06,0.03,0.02,0.06,0.14,0.20,0.13


In [37]:
for el in pg_dict_b:
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.085
Bleu_2: 0.038
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.039
ROUGE_L: 0.076
CIDEr: 0.046
SPICE: 0.127

b
Bleu_1: 0.110
Bleu_2: 0.039
Bleu_3: 0.013
Bleu_4: 0.000
METEOR: 0.054
ROUGE_L: 0.143
CIDEr: 0.201
SPICE: 0.081

c
Bleu_1: 0.051
Bleu_2: 0.011
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.025
ROUGE_L: 0.055
CIDEr: 0.058
SPICE: 0.038

d
Bleu_1: 0.062
Bleu_2: 0.014
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.028
ROUGE_L: 0.039
CIDEr: 0.034
SPICE: 0.026

e
Bleu_1: 0.024
Bleu_2: 0.010
Bleu_3: 0.005
Bleu_4: 0.003
METEOR: 0.019
ROUGE_L: 0.057
CIDEr: 0.043
SPICE: 0.024

f
Bleu_1: 0.098
Bleu_2: 0.039
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.034
ROUGE_L: 0.073
CIDEr: 0.085
SPICE: 0.013

g
Bleu_1: 0.074
Bleu_2: 0.045
Bleu_3: 0.028
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.074
CIDEr: 0.160
SPICE: 0.026

h
Bleu_1: 0.048
Bleu_2: 0.014
Bleu_3: 0.005
Bleu_4: 0.000
METEOR: 0.030
ROUGE_L: 0.080
CIDEr: 0.069
SPICE: 0.050

i
Bleu_1: 0.104
Bleu_2: 0.039
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.036
ROUGE_L: 0.118
C

In [38]:
all_topics_b = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_b.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics_b)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.08,0.04,0.08,0.05,0.13,a
1,0.11,0.05,0.14,0.20,0.08,b
2,0.05,0.03,0.06,0.06,0.04,c
3,0.06,0.03,0.04,0.03,0.03,d
4,0.02,0.02,0.06,0.04,0.02,e
5,0.10,0.03,0.07,0.08,0.01,f
6,0.07,0.03,0.07,0.16,0.03,g
7,0.05,0.03,0.08,0.07,0.05,h
8,0.10,0.04,0.12,0.15,0.02,i
9,0.14,0.06,0.13,0.13,0.07,j


# Fat

In [39]:
df_f1, pg_dict_f = create_dict(df_f)

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [40]:
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_f1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [41]:
t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)

In [42]:
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

In [43]:
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4140, 'reflen': 7635, 'guess': [4140, 3665, 3190, 2715], 'correct': [764, 133, 36, 16]}
ratio: 0.5422396856580822
Bleu_1: 0.079
Bleu_2: 0.035
Bleu_3: 0.018
Bleu_4: 0.011
computing METEOR score...
METEOR: 0.041
computing Rouge score...
ROUGE_L: 0.104
computing CIDEr score...
CIDEr: 0.149
computing SPICE score...
SPICE: 0.064


In [44]:
file_dict_f = file_path_dict(pg_dict_f, test='fat')
web_res_dict_f = dict()

for el in pg_dict_f:
    print(el)
    web_res_dict_f[el] = calc_metrics(file_dict_f, topic=el, anno='o')


all_res_f = []
for el in pg_dict_f:
    all_res_f.append(web_res_dict_f[el])



a
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 97, 'reflen': 128, 'guess': [97, 85, 73, 61], 'correct': [13, 3, 0, 0]}
ratio: 0.7578124999940796
Bleu_1: 0.097
Bleu_2: 0.050
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.045
computing Rouge score...
ROUGE_L: 0.086
computing CIDEr score...
CIDEr: 0.078
computing SPICE score...
SPICE: 0.153
b
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 471, 'reflen': 822, 'guess': [471, 415, 359, 303], 'correct': [119, 22, 5, 1]}
ratio: 0.5729927007292299
Bleu_1: 0.120
Bleu_2: 0.055
Bleu_3: 0.027
Bleu_4: 0.013
computing METEOR score...
METEOR: 0.061
computing

In [45]:
df_res_f = pd.DataFrame(all_res_f)
display(df_res_f.describe().round(2))
#     print(el)
# #     print(split_res_dict[el])
#     for metric, score in web_res_dict_f[el].items():
#         print(f'{metric}: {score:.3f}')
#     print()
all_topics_f = []
for k in web_res_dict_f:
    d_temp = copy.deepcopy(web_res_dict_f[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_f.append(d_temp)
#     break
# d_temp
df_tmp_f = pd.DataFrame(all_topics_f)
display(df_tmp_f.round(2))

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.09,0.04,0.02,0.01,0.04,0.10,0.13,0.06
std,0.04,0.02,0.02,0.01,0.02,0.04,0.08,0.04
min,0.02,0.01,0.00,0.00,0.02,0.04,0.03,0.01
25%,0.06,0.02,0.00,0.00,0.03,0.07,0.08,0.03
50%,0.09,0.05,0.01,0.00,0.04,0.09,0.11,0.04
75%,0.13,0.06,0.04,0.02,0.05,0.13,0.22,0.08
max,0.14,0.08,0.05,0.03,0.07,0.15,0.24,0.15


,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.10,0.04,0.09,0.08,0.15,a
1,0.12,0.06,0.15,0.23,0.09,b
2,0.06,0.03,0.06,0.05,0.04,c
3,0.07,0.03,0.04,0.03,0.03,d
4,0.02,0.02,0.07,0.12,0.03,e
5,0.13,0.04,0.09,0.10,0.01,f
6,0.09,0.03,0.09,0.19,0.04,g
7,0.05,0.04,0.09,0.08,0.06,h
8,0.13,0.05,0.14,0.24,0.03,i
9,0.14,0.07,0.15,0.23,0.10,j


# WIT

In [46]:
df_w1, pg_dict_w = create_dict(df_w)

all_dict = dict()
all_dict['data'] = copy.deepcopy(df_w1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])
t_data = dict()

t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4045, 'reflen': 7635, 'guess': [4045, 3570, 3095, 2620], 'correct': [661, 92, 16, 3]}
ratio: 0.5297969875572325
Bleu_1: 0.067
Bleu_2: 0.027
Bleu_3: 0.011
Bleu_4: 0.005
computing METEOR score...
METEOR: 0.034
computing Rouge score...
ROUGE_L: 0.093
computing CIDEr score...
CIDEr: 0.104
computing SPICE score...
SPICE: 0.050


In [47]:
file_dict_w = file_path_dict(pg_dict_w, test='wit')
web_res_dict_w = dict()

for el in pg_dict_w:
    print(el)
    web_res_dict_w[el] = calc_metrics(file_dict_w, topic=el, anno='o')


all_res_w = []
for el in pg_dict_w:
    all_res_w.append(web_res_dict_w[el])


df_res_w = pd.DataFrame(all_res_w)
display(df_res_w.describe().round(2))
# for el in pg_dict_f:
#     print(el)
# #     print(split_res_dict[el])
#     for metric, score in web_res_dict_f[el].items():
#         print(f'{metric}: {score:.3f}')
#     print()
all_topics_w = []
for k in web_res_dict_w:
    d_temp = copy.deepcopy(web_res_dict_w[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_w.append(d_temp)
#     break
# d_temp
df_tmp_w = pd.DataFrame(all_topics_w)
display(df_tmp_w.round(2))

a
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 104, 'reflen': 128, 'guess': [104, 92, 80, 68], 'correct': [8, 1, 0, 0]}
ratio: 0.8124999999936524
Bleu_1: 0.061
Bleu_2: 0.023
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.027
computing Rouge score...
ROUGE_L: 0.060
computing CIDEr score...
CIDEr: 0.066
computing SPICE score...
SPICE: 0.070
b
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 374, 'reflen': 822, 'guess': [374, 318, 262, 206], 'correct': [88, 14, 4, 0]}
ratio: 0.45498783454932484
Bleu_1: 0.071
Bleu_2: 0.031
Bleu_3: 0.016
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.047
computin

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.08,0.03,0.01,0.00,0.04,0.09,0.11,0.05
std,0.03,0.02,0.01,0.01,0.01,0.03,0.06,0.02
min,0.02,0.01,0.00,0.00,0.02,0.06,0.04,0.02
25%,0.06,0.02,0.00,0.00,0.03,0.07,0.07,0.03
50%,0.08,0.03,0.00,0.00,0.04,0.08,0.08,0.04
75%,0.10,0.05,0.01,0.00,0.04,0.11,0.14,0.07
max,0.12,0.06,0.03,0.02,0.05,0.13,0.20,0.09


,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.06,0.03,0.06,0.07,0.07,a
1,0.07,0.05,0.13,0.18,0.07,b
2,0.08,0.03,0.09,0.04,0.09,c
3,0.10,0.04,0.07,0.06,0.03,d
4,0.02,0.02,0.07,0.07,0.04,e
5,0.08,0.02,0.06,0.09,0.02,f
6,0.09,0.04,0.08,0.14,0.03,g
7,0.04,0.03,0.08,0.07,0.04,h
8,0.11,0.04,0.12,0.20,0.05,i
9,0.12,0.05,0.12,0.14,0.08,j


# Random

In [48]:
df_r1, pg_dict_r = create_dict(df_r)

all_dict = dict()
all_dict['data'] = copy.deepcopy(df_r1)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['annotations'] = web_2anno(all_dict['data'])
t_data = dict()

t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 3847, 'reflen': 7635, 'guess': [3847, 3372, 2897, 2422], 'correct': [706, 131, 30, 10]}
ratio: 0.503863785199672
Bleu_1: 0.069
Bleu_2: 0.032
Bleu_3: 0.016
Bleu_4: 0.009
computing METEOR score...
METEOR: 0.038
computing Rouge score...
ROUGE_L: 0.103
computing CIDEr score...
CIDEr: 0.138
computing SPICE score...
SPICE: 0.058


In [49]:
file_dict_r = file_path_dict(pg_dict_r, test='ran')
web_res_dict_r = dict()

for el in pg_dict_r:
    print(el)
    web_res_dict_r[el] = calc_metrics(file_dict_r, topic=el, anno='o')


all_res_r = []
for el in pg_dict_r:
    all_res_r.append(web_res_dict_r[el])


df_res_r = pd.DataFrame(all_res_r)
display(df_res_r.describe().round(2))
# for el in pg_dict_f:
#     print(el)
# #     print(split_res_dict[el])
#     for metric, score in web_res_dict_f[el].items():
#         print(f'{metric}: {score:.3f}')
#     print()
all_topics_r = []
for k in web_res_dict_r:
    d_temp = copy.deepcopy(web_res_dict_r[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics_r.append(d_temp)
#     break
# d_temp
df_tmp_r = pd.DataFrame(all_topics_r)
display(df_tmp_r.round(2))

a
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 90, 'reflen': 128, 'guess': [90, 78, 66, 54], 'correct': [8, 2, 0, 0]}
ratio: 0.703124999994507
Bleu_1: 0.058
Bleu_2: 0.031
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.028
computing Rouge score...
ROUGE_L: 0.071
computing CIDEr score...
CIDEr: 0.076
computing SPICE score...
SPICE: 0.102
b
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 403, 'reflen': 822, 'guess': [403, 347, 291, 235], 'correct': [103, 17, 3, 0]}
ratio: 0.49026763990208
Bleu_1: 0.090
Bleu_2: 0.040
Bleu_3: 0.018
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.054
computing Rou

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.08,0.04,0.02,0.01,0.04,0.09,0.13,0.05
std,0.04,0.02,0.02,0.01,0.01,0.04,0.09,0.03
min,0.02,0.01,0.00,0.00,0.02,0.03,0.01,0.01
25%,0.05,0.02,0.00,0.00,0.02,0.07,0.07,0.02
50%,0.07,0.04,0.01,0.00,0.04,0.08,0.13,0.05
75%,0.11,0.06,0.04,0.00,0.05,0.13,0.19,0.08
max,0.12,0.07,0.04,0.03,0.06,0.16,0.29,0.10


,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.06,0.03,0.07,0.08,0.10,a
1,0.09,0.05,0.15,0.18,0.09,b
2,0.03,0.02,0.03,0.01,0.01,c
3,0.07,0.02,0.04,0.03,0.02,d
4,0.02,0.02,0.07,0.06,0.03,e
5,0.11,0.04,0.08,0.15,0.02,f
6,0.08,0.03,0.09,0.21,0.05,g
7,0.05,0.04,0.11,0.12,0.06,h
8,0.12,0.05,0.16,0.29,0.07,i
9,0.12,0.06,0.13,0.19,0.09,j


In [53]:
base = df_tmp[['SPICE']]
base.rename(columns={'SPICE': 'Base'}, inplace=True)

fat = df_tmp_f[['SPICE']]
fat.rename(columns={'SPICE': 'App_2'}, inplace=True)

wit = df_tmp_w[['SPICE']]
wit.rename(columns={'SPICE': 'App_1'}, inplace=True)

ran = df_tmp_r[['SPICE']]
ran.rename(columns={'SPICE': 'Random'}, inplace=True)

t = df_tmp_r[['topic']]

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
vertical_concat = pd.concat([t, fat, wit, base, ran], axis=1)
vertical_concat

,topic,App_2,App_1,Base,Random
0,a,0.152658,0.070250,0.126652,0.101588
1,b,0.085103,0.072864,0.080979,0.085688
2,c,0.041185,0.090591,0.037858,0.006734
3,d,0.025235,0.032608,0.026284,0.016226
4,e,0.030624,0.038971,0.024044,0.030834
5,f,0.013714,0.021818,0.013077,0.017622
6,g,0.042654,0.026144,0.025522,0.049560
7,h,0.063534,0.038070,0.049994,0.056200
8,i,0.033130,0.050321,0.022183,0.073032
9,j,0.104834,0.076561,0.070363,0.093914


In [55]:
vertical_concat.to_csv(f'av_spice.csv', index=False)

## human av

In [24]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 108, 'reflen': 65, 'guess': [108, 96, 84, 72], 'correct': [37, 19, 10, 6]}
ratio: 1.6615384615128994
Bleu_1: 0.343
Bleu_2: 0.260
Bleu_3: 0.201
Bleu_4: 0.161
computing METEOR score...
METEOR: 0.256
computing Rouge score...
ROUGE_L: 0.453
computing CIDEr score...
CIDEr: 1.031
computing SPICE score...
SPICE: 0.196
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 495, 'reflen': 346, 'guess': [495, 439, 383, 327], 'correct': [154, 72, 27, 4]}
ratio: 1.4306358381461544
Bleu_1: 0.311
Bleu_2: 0.226
Bleu_3: 0.153
Bleu_4: 0.081
computing METEOR score...
METEOR: 0.210
computing

In [25]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.22,0.15,0.10,0.04,0.18,0.28,0.53,0.08
std,0.09,0.07,0.06,0.05,0.05,0.10,0.27,0.06
min,0.11,0.06,0.04,0.00,0.10,0.16,0.24,0.02
25%,0.15,0.09,0.06,0.01,0.14,0.21,0.33,0.03
50%,0.17,0.12,0.08,0.03,0.17,0.22,0.44,0.05
75%,0.30,0.22,0.15,0.07,0.22,0.36,0.68,0.12
max,0.34,0.26,0.20,0.16,0.26,0.45,1.03,0.20


In [26]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.343
Bleu_2: 0.260
Bleu_3: 0.201
Bleu_4: 0.161
METEOR: 0.256
ROUGE_L: 0.453
CIDEr: 1.031
SPICE: 0.196

b
Bleu_1: 0.311
Bleu_2: 0.226
Bleu_3: 0.153
Bleu_4: 0.081
METEOR: 0.210
ROUGE_L: 0.349
CIDEr: 0.491
SPICE: 0.077

c
Bleu_1: 0.158
Bleu_2: 0.093
Bleu_3: 0.064
Bleu_4: 0.000
METEOR: 0.150
ROUGE_L: 0.207
CIDEr: 0.308
SPICE: 0.026

d
Bleu_1: 0.114
Bleu_2: 0.057
Bleu_3: 0.038
Bleu_4: 0.000
METEOR: 0.102
ROUGE_L: 0.159
CIDEr: 0.243
SPICE: 0.017

e
Bleu_1: 0.151
Bleu_2: 0.091
Bleu_3: 0.056
Bleu_4: 0.026
METEOR: 0.133
ROUGE_L: 0.192
CIDEr: 0.326
SPICE: 0.042

f
Bleu_1: 0.304
Bleu_2: 0.223
Bleu_3: 0.162
Bleu_4: 0.074
METEOR: 0.247
ROUGE_L: 0.361
CIDEr: 0.874
SPICE: 0.154

g
Bleu_1: 0.160
Bleu_2: 0.118
Bleu_3: 0.088
Bleu_4: 0.000
METEOR: 0.142
ROUGE_L: 0.208
CIDEr: 0.737
SPICE: 0.065

h
Bleu_1: 0.171
Bleu_2: 0.116
Bleu_3: 0.077
Bleu_4: 0.026
METEOR: 0.187
ROUGE_L: 0.229
CIDEr: 0.396
SPICE: 0.029

i
Bleu_1: 0.152
Bleu_2: 0.086
Bleu_3: 0.058
Bleu_4: 0.033
METEOR: 0.140
ROUGE_L: 0.210
C

In [27]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.34,0.26,0.45,1.03,0.20,a
1,0.31,0.21,0.35,0.49,0.08,b
2,0.16,0.15,0.21,0.31,0.03,c
3,0.11,0.10,0.16,0.24,0.02,d
4,0.15,0.13,0.19,0.33,0.04,e
5,0.30,0.25,0.36,0.87,0.15,f
6,0.16,0.14,0.21,0.74,0.07,g
7,0.17,0.19,0.23,0.40,0.03,h
8,0.15,0.14,0.21,0.34,0.02,i
9,0.30,0.22,0.38,0.53,0.13,j


## av web

In [28]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

In [29]:
t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)

In [30]:
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

In [31]:
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4341, 'reflen': 7635, 'guess': [4341, 3866, 3391, 2916], 'correct': [655, 82, 13, 3]}
ratio: 0.5685658153240906
Bleu_1: 0.071
Bleu_2: 0.026
Bleu_3: 0.011
Bleu_4: 0.005
computing METEOR score...
METEOR: 0.035
computing Rouge score...
ROUGE_L: 0.090
computing CIDEr score...
CIDEr: 0.108
computing SPICE score...
SPICE: 0.050


In [32]:
web_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    web_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 108, 'reflen': 128, 'guess': [108, 96, 84, 72], 'correct': [11, 2, 0, 0]}
ratio: 0.8437499999934083
Bleu_1: 0.085
Bleu_2: 0.038
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.039
computing Rouge score...
ROUGE_L: 0.076
computing CIDEr score...
CIDEr: 0.046
computing SPICE score...
SPICE: 0.127
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 495, 'reflen': 822, 'guess': [495, 439, 383, 327], 'correct': [105, 12, 1, 0]}
ratio: 0.6021897810211653
Bleu_1: 0.110
Bleu_2: 0.039
Bleu_3: 0.013
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.054
computing R

In [33]:
all_res = []
for el in gk.groups.keys():
    all_res.append(web_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.08,0.03,0.01,0.00,0.04,0.08,0.10,0.05
std,0.03,0.02,0.01,0.00,0.01,0.03,0.06,0.04
min,0.02,0.01,0.00,0.00,0.02,0.04,0.03,0.01
25%,0.05,0.01,0.00,0.00,0.03,0.06,0.05,0.02
50%,0.08,0.04,0.00,0.00,0.03,0.08,0.08,0.03
75%,0.10,0.04,0.01,0.00,0.04,0.11,0.15,0.07
max,0.14,0.06,0.03,0.02,0.06,0.14,0.20,0.13


In [34]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.085
Bleu_2: 0.038
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.039
ROUGE_L: 0.076
CIDEr: 0.046
SPICE: 0.127

b
Bleu_1: 0.110
Bleu_2: 0.039
Bleu_3: 0.013
Bleu_4: 0.000
METEOR: 0.054
ROUGE_L: 0.143
CIDEr: 0.201
SPICE: 0.081

c
Bleu_1: 0.051
Bleu_2: 0.011
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.025
ROUGE_L: 0.055
CIDEr: 0.058
SPICE: 0.038

d
Bleu_1: 0.062
Bleu_2: 0.014
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.028
ROUGE_L: 0.039
CIDEr: 0.034
SPICE: 0.026

e
Bleu_1: 0.024
Bleu_2: 0.010
Bleu_3: 0.005
Bleu_4: 0.003
METEOR: 0.019
ROUGE_L: 0.057
CIDEr: 0.043
SPICE: 0.024

f
Bleu_1: 0.098
Bleu_2: 0.039
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.034
ROUGE_L: 0.073
CIDEr: 0.085
SPICE: 0.013

g
Bleu_1: 0.074
Bleu_2: 0.045
Bleu_3: 0.028
Bleu_4: 0.000
METEOR: 0.035
ROUGE_L: 0.074
CIDEr: 0.160
SPICE: 0.026

h
Bleu_1: 0.048
Bleu_2: 0.014
Bleu_3: 0.005
Bleu_4: 0.000
METEOR: 0.030
ROUGE_L: 0.080
CIDEr: 0.069
SPICE: 0.050

i
Bleu_1: 0.104
Bleu_2: 0.039
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.036
ROUGE_L: 0.118
C

In [35]:
all_topics = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.08,0.04,0.08,0.05,0.13,a
1,0.11,0.05,0.14,0.20,0.08,b
2,0.05,0.03,0.06,0.06,0.04,c
3,0.06,0.03,0.04,0.03,0.03,d
4,0.02,0.02,0.06,0.04,0.02,e
5,0.10,0.03,0.07,0.08,0.01,f
6,0.07,0.03,0.07,0.16,0.03,g
7,0.05,0.03,0.08,0.07,0.05,h
8,0.10,0.04,0.12,0.15,0.02,i
9,0.14,0.06,0.13,0.13,0.07,j


# av fat human

In [20]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = h_gt2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4094, 'reflen': 2266, 'guess': [4094, 3619, 3144, 2669], 'correct': [792, 291, 109, 15]}
ratio: 1.8067078552507472
Bleu_1: 0.193
Bleu_2: 0.125
Bleu_3: 0.081
Bleu_4: 0.042
computing METEOR score...
METEOR: 0.149
computing Rouge score...
ROUGE_L: 0.241
computing CIDEr score...
CIDEr: 0.475
computing SPICE score...
SPICE: 0.078


In [21]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')



loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 91, 'reflen': 65, 'guess': [91, 79, 67, 55], 'correct': [35, 19, 10, 6]}
ratio: 1.3999999999784614
Bleu_1: 0.385
Bleu_2: 0.304
Bleu_3: 0.240
Bleu_4: 0.197
computing METEOR score...
METEOR: 0.258
computing Rouge score...
ROUGE_L: 0.464
computing CIDEr score...
CIDEr: 2.201
computing SPICE score...
SPICE: 0.349
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 414, 'reflen': 346, 'guess': [414, 358, 302, 246], 'correct': [131, 59, 28, 3]}
ratio: 1.1965317919040563
Bleu_1: 0.316
Bleu_2: 0.228
Bleu_3: 0.169
Bleu_4: 0.088
computing METEOR score...
METEOR: 0.195
computing R

In [22]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.21,0.14,0.10,0.05,0.16,0.26,0.63,0.10
std,0.09,0.08,0.06,0.06,0.05,0.10,0.57,0.10
min,0.12,0.06,0.04,0.00,0.10,0.17,0.29,0.01
25%,0.16,0.09,0.06,0.00,0.12,0.19,0.33,0.04
50%,0.16,0.10,0.07,0.03,0.14,0.21,0.43,0.06
75%,0.28,0.18,0.11,0.07,0.19,0.32,0.62,0.12
max,0.38,0.30,0.24,0.20,0.26,0.46,2.20,0.35


In [23]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.385
Bleu_2: 0.304
Bleu_3: 0.240
Bleu_4: 0.197
METEOR: 0.258
ROUGE_L: 0.464
CIDEr: 2.201
SPICE: 0.349

b
Bleu_1: 0.316
Bleu_2: 0.228
Bleu_3: 0.169
Bleu_4: 0.088
METEOR: 0.195
ROUGE_L: 0.329
CIDEr: 0.469
SPICE: 0.038

c
Bleu_1: 0.160
Bleu_2: 0.102
Bleu_3: 0.067
Bleu_4: 0.034
METEOR: 0.150
ROUGE_L: 0.218
CIDEr: 0.351
SPICE: 0.054

d
Bleu_1: 0.124
Bleu_2: 0.065
Bleu_3: 0.044
Bleu_4: 0.000
METEOR: 0.099
ROUGE_L: 0.170
CIDEr: 0.325
SPICE: 0.024

e
Bleu_1: 0.158
Bleu_2: 0.098
Bleu_3: 0.064
Bleu_4: 0.030
METEOR: 0.125
ROUGE_L: 0.194
CIDEr: 0.383
SPICE: 0.069

f
Bleu_1: 0.242
Bleu_2: 0.171
Bleu_3: 0.117
Bleu_4: 0.076
METEOR: 0.184
ROUGE_L: 0.299
CIDEr: 0.725
SPICE: 0.131

g
Bleu_1: 0.163
Bleu_2: 0.105
Bleu_3: 0.069
Bleu_4: 0.000
METEOR: 0.118
ROUGE_L: 0.186
CIDEr: 0.611
SPICE: 0.063

h
Bleu_1: 0.134
Bleu_2: 0.081
Bleu_3: 0.053
Bleu_4: 0.000
METEOR: 0.118
ROUGE_L: 0.174
CIDEr: 0.292
SPICE: 0.039

i
Bleu_1: 0.156
Bleu_2: 0.084
Bleu_3: 0.054
Bleu_4: 0.000
METEOR: 0.120
ROUGE_L: 0.204
C

In [24]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.38,0.26,0.46,2.20,0.35,a
1,0.32,0.19,0.33,0.47,0.04,b
2,0.16,0.15,0.22,0.35,0.05,c
3,0.12,0.10,0.17,0.33,0.02,d
4,0.16,0.13,0.19,0.38,0.07,e
5,0.24,0.18,0.30,0.73,0.13,f
6,0.16,0.12,0.19,0.61,0.06,g
7,0.13,0.12,0.17,0.29,0.04,h
8,0.16,0.12,0.20,0.31,0.01,i
9,0.29,0.19,0.37,0.63,0.19,j


# av WIT human

In [20]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = h_gt2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4171, 'reflen': 2266, 'guess': [4171, 3696, 3221, 2746], 'correct': [787, 248, 84, 10]}
ratio: 1.8406884377750041
Bleu_1: 0.189
Bleu_2: 0.113
Bleu_3: 0.069
Bleu_4: 0.033
computing METEOR score...
METEOR: 0.144
computing Rouge score...
ROUGE_L: 0.236
computing CIDEr score...
CIDEr: 0.402
computing SPICE score...
SPICE: 0.094


In [21]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='s')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 104, 'reflen': 65, 'guess': [104, 92, 80, 68], 'correct': [32, 15, 6, 3]}
ratio: 1.5999999999753844
Bleu_1: 0.308
Bleu_2: 0.224
Bleu_3: 0.156
Bleu_4: 0.114
computing METEOR score...
METEOR: 0.206
computing Rouge score...
ROUGE_L: 0.401
computing CIDEr score...
CIDEr: 1.016
computing SPICE score...
SPICE: 0.225
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 442, 'reflen': 346, 'guess': [442, 386, 330, 274], 'correct': [130, 58, 24, 1]}
ratio: 1.2774566473951519
Bleu_1: 0.294
Bleu_2: 0.210
Bleu_3: 0.148
Bleu_4: 0.059
computing METEOR score...
METEOR: 0.187
computing 

In [22]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.20,0.12,0.08,0.04,0.15,0.25,0.47,0.12
std,0.07,0.06,0.05,0.04,0.04,0.08,0.27,0.06
min,0.13,0.06,0.03,0.00,0.10,0.17,0.22,0.04
25%,0.14,0.06,0.04,0.00,0.12,0.18,0.29,0.07
50%,0.18,0.10,0.06,0.03,0.14,0.24,0.36,0.11
75%,0.27,0.17,0.10,0.05,0.18,0.31,0.56,0.15
max,0.31,0.22,0.16,0.11,0.21,0.40,1.02,0.22


In [23]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.308
Bleu_2: 0.224
Bleu_3: 0.156
Bleu_4: 0.114
METEOR: 0.206
ROUGE_L: 0.401
CIDEr: 1.016
SPICE: 0.225

b
Bleu_1: 0.294
Bleu_2: 0.210
Bleu_3: 0.148
Bleu_4: 0.059
METEOR: 0.187
ROUGE_L: 0.320
CIDEr: 0.394
SPICE: 0.059

c
Bleu_1: 0.138
Bleu_2: 0.055
Bleu_3: 0.030
Bleu_4: 0.000
METEOR: 0.112
ROUGE_L: 0.178
CIDEr: 0.218
SPICE: 0.101

d
Bleu_1: 0.160
Bleu_2: 0.059
Bleu_3: 0.031
Bleu_4: 0.020
METEOR: 0.126
ROUGE_L: 0.217
CIDEr: 0.293
SPICE: 0.135

e
Bleu_1: 0.142
Bleu_2: 0.084
Bleu_3: 0.049
Bleu_4: 0.024
METEOR: 0.118
ROUGE_L: 0.184
CIDEr: 0.325
SPICE: 0.108

f
Bleu_1: 0.233
Bleu_2: 0.144
Bleu_3: 0.091
Bleu_4: 0.048
METEOR: 0.148
ROUGE_L: 0.273
CIDEr: 0.561
SPICE: 0.109

g
Bleu_1: 0.196
Bleu_2: 0.120
Bleu_3: 0.079
Bleu_4: 0.047
METEOR: 0.179
ROUGE_L: 0.256
CIDEr: 0.850
SPICE: 0.172

h
Bleu_1: 0.139
Bleu_2: 0.076
Bleu_3: 0.047
Bleu_4: 0.000
METEOR: 0.116
ROUGE_L: 0.178
CIDEr: 0.291
SPICE: 0.047

i
Bleu_1: 0.131
Bleu_2: 0.060
Bleu_3: 0.033
Bleu_4: 0.000
METEOR: 0.105
ROUGE_L: 0.175
C

In [24]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.31,0.21,0.40,1.02,0.22,a
1,0.29,0.19,0.32,0.39,0.06,b
2,0.14,0.11,0.18,0.22,0.10,c
3,0.16,0.13,0.22,0.29,0.14,d
4,0.14,0.12,0.18,0.32,0.11,e
5,0.23,0.15,0.27,0.56,0.11,f
6,0.20,0.18,0.26,0.85,0.17,g
7,0.14,0.12,0.18,0.29,0.05,h
8,0.13,0.10,0.17,0.26,0.04,i
9,0.28,0.19,0.34,0.54,0.16,j


# wit av cider web

In [20]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4281, 'reflen': 7635, 'guess': [4281, 3806, 3331, 2856], 'correct': [763, 142, 42, 19]}
ratio: 0.5607072691551328
Bleu_1: 0.081
Bleu_2: 0.037
Bleu_3: 0.020
Bleu_4: 0.012
computing METEOR score...
METEOR: 0.041
computing Rouge score...
ROUGE_L: 0.107
computing CIDEr score...
CIDEr: 0.157
computing SPICE score...
SPICE: 0.060


In [21]:
web_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    web_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 111, 'reflen': 128, 'guess': [111, 99, 87, 75], 'correct': [10, 2, 0, 0]}
ratio: 0.8671874999932252
Bleu_1: 0.077
Bleu_2: 0.037
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.033
computing Rouge score...
ROUGE_L: 0.072
computing CIDEr score...
CIDEr: 0.078
computing SPICE score...
SPICE: 0.091
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 442, 'reflen': 822, 'guess': [442, 386, 330, 274], 'correct': [119, 24, 7, 2]}
ratio: 0.5377128953764748
Bleu_1: 0.114
Bleu_2: 0.055
Bleu_3: 0.030
Bleu_4: 0.017
computing METEOR score...
METEOR: 0.063
computing R

In [22]:
all_res = []
for el in gk.groups.keys():
    all_res.append(web_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.09,0.05,0.02,0.01,0.04,0.10,0.15,0.05
std,0.04,0.03,0.02,0.02,0.01,0.04,0.08,0.03
min,0.02,0.01,0.00,0.00,0.02,0.05,0.05,0.03
25%,0.06,0.02,0.01,0.00,0.03,0.07,0.08,0.03
50%,0.09,0.05,0.02,0.00,0.04,0.09,0.15,0.05
75%,0.12,0.06,0.04,0.02,0.05,0.13,0.22,0.08
max,0.15,0.10,0.06,0.04,0.06,0.17,0.27,0.10


In [23]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.077
Bleu_2: 0.037
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.033
ROUGE_L: 0.072
CIDEr: 0.078
SPICE: 0.091

b
Bleu_1: 0.114
Bleu_2: 0.055
Bleu_3: 0.030
Bleu_4: 0.017
METEOR: 0.063
ROUGE_L: 0.168
CIDEr: 0.255
SPICE: 0.093

c
Bleu_1: 0.056
Bleu_2: 0.016
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.024
ROUGE_L: 0.053
CIDEr: 0.054
SPICE: 0.030

d
Bleu_1: 0.083
Bleu_2: 0.032
Bleu_3: 0.015
Bleu_4: 0.000
METEOR: 0.033
ROUGE_L: 0.061
CIDEr: 0.079
SPICE: 0.028

e
Bleu_1: 0.025
Bleu_2: 0.010
Bleu_3: 0.005
Bleu_4: 0.003
METEOR: 0.021
ROUGE_L: 0.070
CIDEr: 0.129
SPICE: 0.029

f
Bleu_1: 0.151
Bleu_2: 0.096
Bleu_3: 0.060
Bleu_4: 0.035
METEOR: 0.055
ROUGE_L: 0.123
CIDEr: 0.269
SPICE: 0.027

g
Bleu_1: 0.097
Bleu_2: 0.060
Bleu_3: 0.043
Bleu_4: 0.026
METEOR: 0.043
ROUGE_L: 0.090
CIDEr: 0.234
SPICE: 0.046

h
Bleu_1: 0.058
Bleu_2: 0.020
Bleu_3: 0.007
Bleu_4: 0.000
METEOR: 0.036
ROUGE_L: 0.099
CIDEr: 0.086
SPICE: 0.056

i
Bleu_1: 0.117
Bleu_2: 0.062
Bleu_3: 0.025
Bleu_4: 0.000
METEOR: 0.042
ROUGE_L: 0.129
C

In [24]:
all_topics = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.08,0.03,0.07,0.08,0.09,a
1,0.11,0.06,0.17,0.25,0.09,b
2,0.06,0.02,0.05,0.05,0.03,c
3,0.08,0.03,0.06,0.08,0.03,d
4,0.02,0.02,0.07,0.13,0.03,e
5,0.15,0.05,0.12,0.27,0.03,f
6,0.10,0.04,0.09,0.23,0.05,g
7,0.06,0.04,0.10,0.09,0.06,h
8,0.12,0.04,0.13,0.16,0.05,i
9,0.15,0.06,0.14,0.19,0.10,j


# fat av cider web

In [20]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4281, 'reflen': 7635, 'guess': [4281, 3806, 3331, 2856], 'correct': [763, 142, 42, 19]}
ratio: 0.5607072691551328
Bleu_1: 0.081
Bleu_2: 0.037
Bleu_3: 0.020
Bleu_4: 0.012
computing METEOR score...
METEOR: 0.041
computing Rouge score...
ROUGE_L: 0.107
computing CIDEr score...
CIDEr: 0.157
computing SPICE score...
SPICE: 0.060


In [21]:
web_res_dict = dict()
for el in gk.groups.keys():
#     split_res_dict[el] = calc_metrics(topic=el, anno='s')
#     con_res_dict[el] = calc_metrics(topic=el, anno='c')
    web_res_dict[el] = calc_metrics(topic=el, anno='o')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 111, 'reflen': 128, 'guess': [111, 99, 87, 75], 'correct': [10, 2, 0, 0]}
ratio: 0.8671874999932252
Bleu_1: 0.077
Bleu_2: 0.037
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.033
computing Rouge score...
ROUGE_L: 0.072
computing CIDEr score...
CIDEr: 0.078
computing SPICE score...
SPICE: 0.091
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 442, 'reflen': 822, 'guess': [442, 386, 330, 274], 'correct': [119, 24, 7, 2]}
ratio: 0.5377128953764748
Bleu_1: 0.114
Bleu_2: 0.055
Bleu_3: 0.030
Bleu_4: 0.017
computing METEOR score...
METEOR: 0.063
computing R

In [22]:
all_res = []
for el in gk.groups.keys():
    all_res.append(web_res_dict[el])
df_res = pd.DataFrame(all_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.09,0.05,0.02,0.01,0.04,0.10,0.15,0.05
std,0.04,0.03,0.02,0.02,0.01,0.04,0.08,0.03
min,0.02,0.01,0.00,0.00,0.02,0.05,0.05,0.03
25%,0.06,0.02,0.01,0.00,0.03,0.07,0.08,0.03
50%,0.09,0.05,0.02,0.00,0.04,0.09,0.15,0.05
75%,0.12,0.06,0.04,0.02,0.05,0.13,0.22,0.08
max,0.15,0.10,0.06,0.04,0.06,0.17,0.27,0.10


In [23]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in web_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.077
Bleu_2: 0.037
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.033
ROUGE_L: 0.072
CIDEr: 0.078
SPICE: 0.091

b
Bleu_1: 0.114
Bleu_2: 0.055
Bleu_3: 0.030
Bleu_4: 0.017
METEOR: 0.063
ROUGE_L: 0.168
CIDEr: 0.255
SPICE: 0.093

c
Bleu_1: 0.056
Bleu_2: 0.016
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.024
ROUGE_L: 0.053
CIDEr: 0.054
SPICE: 0.030

d
Bleu_1: 0.083
Bleu_2: 0.032
Bleu_3: 0.015
Bleu_4: 0.000
METEOR: 0.033
ROUGE_L: 0.061
CIDEr: 0.079
SPICE: 0.028

e
Bleu_1: 0.025
Bleu_2: 0.010
Bleu_3: 0.005
Bleu_4: 0.003
METEOR: 0.021
ROUGE_L: 0.070
CIDEr: 0.129
SPICE: 0.029

f
Bleu_1: 0.151
Bleu_2: 0.096
Bleu_3: 0.060
Bleu_4: 0.035
METEOR: 0.055
ROUGE_L: 0.123
CIDEr: 0.269
SPICE: 0.027

g
Bleu_1: 0.097
Bleu_2: 0.060
Bleu_3: 0.043
Bleu_4: 0.026
METEOR: 0.043
ROUGE_L: 0.090
CIDEr: 0.234
SPICE: 0.046

h
Bleu_1: 0.058
Bleu_2: 0.020
Bleu_3: 0.007
Bleu_4: 0.000
METEOR: 0.036
ROUGE_L: 0.099
CIDEr: 0.086
SPICE: 0.056

i
Bleu_1: 0.117
Bleu_2: 0.062
Bleu_3: 0.025
Bleu_4: 0.000
METEOR: 0.042
ROUGE_L: 0.129
C

In [24]:
all_topics = []
for k in web_res_dict:
    d_temp = copy.deepcopy(web_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.08,0.03,0.07,0.08,0.09,a
1,0.11,0.06,0.17,0.25,0.09,b
2,0.06,0.02,0.05,0.05,0.03,c
3,0.08,0.03,0.06,0.08,0.03,d
4,0.02,0.02,0.07,0.13,0.03,e
5,0.15,0.05,0.12,0.27,0.03,f
6,0.10,0.04,0.09,0.23,0.05,g
7,0.06,0.04,0.10,0.09,0.06,h
8,0.12,0.04,0.13,0.16,0.05,i
9,0.15,0.06,0.14,0.19,0.10,j


# Web Sim

In [20]:
# gk = df_wp.groupby('topics')
# pg_dict = dict()
# for el in gk.groups.keys():
all_dict = dict()
all_dict['data'] = copy.deepcopy(df_n)
all_dict['data'] = all_dict['data'].reset_index(drop=True)
temp = all_dict['data'][['image_id', 'ofa_caption']]
temp.columns = temp.columns.str.replace('ofa_caption', 'caption')
all_dict['results'] = temp
all_dict['images'] = id2imdf(all_dict['data'])
all_dict['human_annotations'] = web_2anno(all_dict['data'])

t_data = dict()
t_data['info'] = jdata['info']
t_data['images'] = all_dict['images'].to_dict('records')
t_data['type'] = jdata['type']
t_data['licenses'] = jdata['licenses']
t_data['annotations'] = all_dict['human_annotations'].to_dict('records')
file_p = f"data/all_anno.json"
with open(file_p, "w") as final:
    json.dump(t_data, final)
    
res = all_dict['results'].to_dict('records')
file_p = f"data/all_results.json"
with open(file_p, "w") as final:
    json.dump(res, final)

    
annotation_file = 'data/all_anno.json'
results_file = 'data/all_results.json'

# create coco object and coco_result object
coco = COCO(annotation_file)
coco_result = coco.loadRes(results_file)

# create coco_eval object by taking coco and coco_result
coco_eval = COCOEvalCap(coco, coco_result)

# evaluate on a subset of images by setting
# coco_eval.params['image_id'] = coco_result.getImgIds()
# please remove this line when evaluating the full validation set
coco_eval.params['image_id'] = coco_result.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
coco_eval.evaluate()

# # print output evaluation scores
# for metric, score in coco_eval.eval.items():
#     print(f'{metric}: {score:.3f}')

/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/test/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying t

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 4140, 'reflen': 7635, 'guess': [4140, 3665, 3190, 2715], 'correct': [764, 133, 36, 16]}
ratio: 0.5422396856580822
Bleu_1: 0.079
Bleu_2: 0.035
Bleu_3: 0.018
Bleu_4: 0.011
computing METEOR score...
METEOR: 0.041
computing Rouge score...
ROUGE_L: 0.104
computing CIDEr score...
CIDEr: 0.149
computing SPICE score...
SPICE: 0.064


In [21]:
human_res_dict = dict()
for el in gk.groups.keys():
    human_res_dict[el] = calc_metrics(topic=el, anno='o')



loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 97, 'reflen': 128, 'guess': [97, 85, 73, 61], 'correct': [13, 3, 0, 0]}
ratio: 0.7578124999940796
Bleu_1: 0.097
Bleu_2: 0.050
Bleu_3: 0.000
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.045
computing Rouge score...
ROUGE_L: 0.086
computing CIDEr score...
CIDEr: 0.078
computing SPICE score...
SPICE: 0.153
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 471, 'reflen': 822, 'guess': [471, 415, 359, 303], 'correct': [119, 22, 5, 1]}
ratio: 0.5729927007292299
Bleu_1: 0.120
Bleu_2: 0.055
Bleu_3: 0.027
Bleu_4: 0.013
computing METEOR score...
METEOR: 0.061
computing Rou

In [22]:
h_res = []
for el in gk.groups.keys():
    h_res.append(human_res_dict[el])
df_res = pd.DataFrame(h_res)
df_res.describe().round(2)

,Bleu_1,Bleu_2,Bleu_3,Bleu_4,METEOR,ROUGE_L,CIDEr,SPICE
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,0.09,0.04,0.02,0.01,0.04,0.10,0.13,0.06
std,0.04,0.02,0.02,0.01,0.02,0.04,0.08,0.04
min,0.02,0.01,0.00,0.00,0.02,0.04,0.03,0.01
25%,0.06,0.02,0.00,0.00,0.03,0.07,0.08,0.03
50%,0.09,0.05,0.01,0.00,0.04,0.09,0.11,0.04
75%,0.13,0.06,0.04,0.02,0.05,0.13,0.22,0.08
max,0.14,0.08,0.05,0.03,0.07,0.15,0.24,0.15


In [23]:
for el in gk.groups.keys():
    print(el)
#     print(split_res_dict[el])
    for metric, score in human_res_dict[el].items():
        print(f'{metric}: {score:.3f}')
    print()

a
Bleu_1: 0.097
Bleu_2: 0.050
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.045
ROUGE_L: 0.086
CIDEr: 0.078
SPICE: 0.153

b
Bleu_1: 0.120
Bleu_2: 0.055
Bleu_3: 0.027
Bleu_4: 0.013
METEOR: 0.061
ROUGE_L: 0.151
CIDEr: 0.227
SPICE: 0.085

c
Bleu_1: 0.059
Bleu_2: 0.016
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.027
ROUGE_L: 0.058
CIDEr: 0.054
SPICE: 0.041

d
Bleu_1: 0.067
Bleu_2: 0.025
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.025
ROUGE_L: 0.044
CIDEr: 0.031
SPICE: 0.025

e
Bleu_1: 0.023
Bleu_2: 0.009
Bleu_3: 0.005
Bleu_4: 0.004
METEOR: 0.021
ROUGE_L: 0.068
CIDEr: 0.116
SPICE: 0.031

f
Bleu_1: 0.129
Bleu_2: 0.057
Bleu_3: 0.000
Bleu_4: 0.000
METEOR: 0.043
ROUGE_L: 0.087
CIDEr: 0.100
SPICE: 0.014

g
Bleu_1: 0.085
Bleu_2: 0.052
Bleu_3: 0.039
Bleu_4: 0.024
METEOR: 0.035
ROUGE_L: 0.086
CIDEr: 0.192
SPICE: 0.043

h
Bleu_1: 0.052
Bleu_2: 0.018
Bleu_3: 0.006
Bleu_4: 0.000
METEOR: 0.036
ROUGE_L: 0.093
CIDEr: 0.077
SPICE: 0.064

i
Bleu_1: 0.130
Bleu_2: 0.072
Bleu_3: 0.044
Bleu_4: 0.030
METEOR: 0.047
ROUGE_L: 0.144
C

In [24]:
all_topics = []
for k in human_res_dict:
    d_temp = copy.deepcopy(human_res_dict[k])
    del d_temp['Bleu_4']
    del d_temp['Bleu_2']
    del d_temp['Bleu_3']
    d_temp['topic'] = k
    all_topics.append(d_temp)
#     break
# d_temp
df_tmp = pd.DataFrame(all_topics)
df_tmp.round(2)

,Bleu_1,METEOR,ROUGE_L,CIDEr,SPICE,topic
0,0.10,0.04,0.09,0.08,0.15,a
1,0.12,0.06,0.15,0.23,0.09,b
2,0.06,0.03,0.06,0.05,0.04,c
3,0.07,0.03,0.04,0.03,0.03,d
4,0.02,0.02,0.07,0.12,0.03,e
5,0.13,0.04,0.09,0.10,0.01,f
6,0.09,0.03,0.09,0.19,0.04,g
7,0.05,0.04,0.09,0.08,0.06,h
8,0.13,0.05,0.14,0.24,0.03,i
9,0.14,0.07,0.15,0.23,0.10,j
